## Практическая работа №8
## Прогнозирование случайных чисел

In [44]:
import random
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import Dense, SimpleRNN, Input
from tensorflow.keras.utils import to_categorical

### В этом задании нам нужно сделать две сети, которые будут угадывать следующее за 10 предыдущими зачение, значения будем генерировать генератором случайных чисел, таким образом, если сеть не научится предсказывать значения, можно будет предположит, что генератор случайных значений работает хорошо
### Первая сеть будет обучаться и предсказывать значения от 0 до 9, начнем со сбора данных

In [62]:
data = [random.randint(0, 9) for i in range(100000)]

X = []
for i in range(len(data) - 11):
    X.append(data[i:i+10])
X = np.array(X, dtype=np.float32)

y = []
for i in range(len(data) - 11):
    y.append(data[i+10])
y = np.array(y, dtype=np.float32)
    
print(f'Первые десять цифр X[0] = {X[0]} и следующая за ними y[0] = {y[0]}')


y = to_categorical(y, 10)

X_train = np.expand_dims( X[:80000], axis=1 )
X_test = np.expand_dims( X[80000:], axis=1 )

y_train = y[:80000]
y_test = y[80000:]

Первые десять цифр X[0] = [6. 4. 4. 1. 2. 0. 6. 8. 1. 6.] и следующая за ними y[0] = 6.0


### Для работы сети будем использовать рекурентный слой SimpleRNN, который принимает несколько значений по очереди и учится предсказывать их последовательность
### В качестве выходного слоя используем Dense, состоящий из 10 нейронов с функцией активации softmax

In [63]:
model = keras.Sequential()
model.add( SimpleRNN(10, activation='relu') )
model.add( Dense(10, activation='softmax') )

model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics='accuracy'
)

model.fit(
    x=X_train,
    y=y_train,
    epochs=5,
    batch_size=32,
    validation_split=0.2
)

Epoch 1/5
2000/2000 [==============================] - 6s 3ms/step - loss: 2.4849 - accuracy: 0.1008 - val_loss: 2.3032 - val_accuracy: 0.1021
Epoch 2/5
2000/2000 [==============================] - 5s 3ms/step - loss: 2.3029 - accuracy: 0.0995 - val_loss: 2.3026 - val_accuracy: 0.1021
Epoch 3/5
2000/2000 [==============================] - 5s 3ms/step - loss: 2.3029 - accuracy: 0.0992 - val_loss: 2.3031 - val_accuracy: 0.1020
Epoch 4/5
2000/2000 [==============================] - 5s 3ms/step - loss: 2.3027 - accuracy: 0.1020 - val_loss: 2.3027 - val_accuracy: 0.1008
Epoch 5/5
2000/2000 [==============================] - 5s 3ms/step - loss: 2.3027 - accuracy: 0.1010 - val_loss: 2.3028 - val_accuracy: 0.1014


### Попробуем предсказать следующие цифры, что бы оценивать уданость прогноза воспользуемся следующей формулой
# $$ \varepsilon = \frac{\overset{n}{\underset{i=1}{\sum}} \mid y_i - y_i^{модели} \mid}{\overset{n}{\underset{i=1}{\sum}} y_i} $$
### Чем эта метрика меньше, тем лучше наша модель предсказывает данные

In [71]:
predict = model.predict(X_test)

metric = sum(abs(y_test - predict)) / sum(y_test)
print(f'Метрика = {np.mean(metric)}')

Метрика = 1.8003177642822266


### Можно сделать вывод, что наша сеть довольно плохо предсказывает новые значения
### Теперь сделаем все то же самое но со случайными цифрами от 0 до 99
### Плюс используем другу архитектуру сети, а сделаем 3 внутренних слоя SimpleRNN, оптимизатор 'Adam' и 7 эпох

In [77]:
data = [random.randint(0, 99) for i in range(100000)]

X = []
for i in range(len(data) - 11):
    X.append(data[i:i+10])
X = np.array(X, dtype=np.float32)

y = []
for i in range(len(data) - 11):
    y.append(data[i+10])
y = np.array(y, dtype=np.float32)
    
print(f'Первые десять цифр X[0] = {X[0]} и следующая за ними y[0] = {y[0]}')


y = to_categorical(y, 100)

X_train = np.expand_dims( X[:80000], axis=1 )
X_test = np.expand_dims( X[80000:], axis=1 )

y_train = y[:80000]
y_test = y[80000:]

Первые десять цифр X[0] = [42. 58. 76. 32.  4. 51. 24. 17. 51. 36.] и следующая за ними y[0] = 44.0


In [81]:
model = keras.Sequential()
model.add( SimpleRNN(100, activation='relu', return_sequences=True) ),
model.add( SimpleRNN(100, activation='relu', return_sequences=True) )
model.add( SimpleRNN(100, activation='relu') )
model.add( Dense(100, activation='softmax') )

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics='accuracy'
)

model.fit(
    x=X_train,
    y=y_train,
    epochs=7,
    batch_size=32,
    validation_split=0.2
)

Epoch 1/7
2000/2000 [==============================] - 11s 5ms/step - loss: 4.6553 - accuracy: 0.0102 - val_loss: 4.6052 - val_accuracy: 0.0105
Epoch 2/7
2000/2000 [==============================] - 9s 5ms/step - loss: 4.6053 - accuracy: 0.0111 - val_loss: 4.6053 - val_accuracy: 0.0097
Epoch 3/7
2000/2000 [==============================] - 9s 5ms/step - loss: 4.6051 - accuracy: 0.0101 - val_loss: 4.6055 - val_accuracy: 0.0097
Epoch 4/7
2000/2000 [==============================] - 9s 5ms/step - loss: 4.6051 - accuracy: 0.0101 - val_loss: 4.6055 - val_accuracy: 0.0097
Epoch 5/7
2000/2000 [==============================] - 9s 5ms/step - loss: 4.6053 - accuracy: 0.0104 - val_loss: 4.6054 - val_accuracy: 0.0105
Epoch 6/7
2000/2000 [==============================] - 10s 5ms/step - loss: 4.6051 - accuracy: 0.0098 - val_loss: 4.6054 - val_accuracy: 0.0105
Epoch 7/7
2000/2000 [==============================] - 9s 5ms/step - loss: 4.6051 - accuracy: 0.0101 - val_loss: 4.6054 - val_accuracy: 0.01

In [82]:
predict = model.predict(X_test)

metric = sum(abs(y_test - predict)) / sum(y_test)
print(f'Метрика = {np.mean(metric)}')

Метрика = 1.9853065013885498


### Даже с увеличинным количеством внутренних слоев и 7 эпохами модель показало плохую предсказательную способность из чего можно сделать вывод, что ей не удается найти паттерны появления чисел в последовательности
### Нейросеть не может предсказать рандомно сгенерированные числа на основе предыдущих рандомно сгенерированных чисел